## Table of Contents
- [Model](#Model)
  - [XGBoost (eXtreme Gradient Boosting)](#XGBoost-%28eXtreme-Gradient-Boosting%29)
  - [Concept](#Concept)
  - [Key Features](#Key-Features)
  - [Hyperparameters](#Hyperparameters)
  - [Applications](#Applications)
- [Implementation](#Implementation)
  - [XGBoost](#XGBoost)
- [🏠 Home](../../../../welcomePage.ipynb)

# Model

## XGBoost (eXtreme Gradient Boosting)

**XGBoost (eXtreme Gradient Boosting)** is an advanced implementation of gradient boosting that is specifically designed to be highly efficient, flexible, and portable. It implements machine learning algorithms under the Gradient Boosting framework. XGBoost provides a parallel tree boosting (also known as GBDT, GBM) that solve many data science problems in a fast and accurate way.

## Concept

The core XGBoost algorithm is an ensemble method that sequentially builds models, and it corrects its own mistakes in the previous steps, enhancing the performance of the ensemble. XGBoost improves upon the base Gradient Boosting Machine (GBM) framework through systems optimization and algorithmic enhancements.

## Key Features

- **Regularization:** Includes both L1 and L2 regularization which helps in reducing overfitting.
- **Handling Sparse Data:** Has built-in handling of missing data values.
- **Tree Pruning:** XGBoost makes splits up to the max depth specified and then starts pruning the tree backwards and removes splits beyond which there is no positive gain.
- **Built-in Cross-Validation:** XGboost allows user to run a cross-validation at each iteration of the boosting process.

## Hyperparameters

Some of the key hyperparameters in XGBoost include:
- **learning_rate:** Step size shrinkage used to prevent overfitting. Range is [0,1].
- **max_depth:** Determines how deeply each tree is allowed to grow during any boosting round.
- **subsample:** Percentage of samples used per tree. Low value can lead to underfitting.
- **colsample_bytree:** Percentage of features used per tree. High value can lead to overfitting.
- **n_estimators:** Number of trees you want to build.
- **objective:** Specifies the learning task and the corresponding learning objective or a custom objective function to be used.

## Applications

XGBoost has become the algorithm of choice for many winning teams of machine learning competitions. It has been successfully used in:
- Predictive modeling competitions,
- Ranking problems like advertisement ranking,
- Classical classification and regression problems.


# Implementation

### Import Libraries

**Press ▶ to visualize the decision boundary.**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

print("Libraries are imported.")

### Import data

**Press ▶ to load the data.**

In [ ]:
import os
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output

# List all .csv and Excel files in the current directory
supported_extensions = ['.csv', '.xlsx', '.xls']
files = [f for f in os.listdir('./data/') if any(f.endswith(ext) for ext in supported_extensions)]

# Create a dropdown widget
dropdown = widgets.Dropdown(
    options=files,
    description='Files:',
    disabled=False,
)

# Create a button widget
button = widgets.Button(
    description='Select',
    disabled=False,
    button_style='',
    tooltip='Click to select file',
    icon='check'
)

# Output widget to display messages
output = widgets.Output()

# Function to handle button click
def on_button_click(b):
    with output:
        clear_output()
        selected_file = dropdown.value
        global data
        if selected_file.endswith('.csv'):
            data = pd.read_csv('./data/'+selected_file)
        elif selected_file.endswith(('.xlsx', '.xls')):
            data = pd.read_excel('./data/'+selected_file)
        print(f"File '{selected_file}' uploaded as data.")

# Attach the function to the button widget
button.on_click(on_button_click)

# Display the dropdown, button widgets, and initial message within the output widget
with output:
    print("Please select a file from the dropdown and click 'Select'.")
display(output)
display(dropdown)
display(button)

**Press ▶ to display the data.**

In [ ]:
display(data.head())
print("Loading pair plot, please wait...")
sns.pairplot(data=data,hue=data.columns[-1],diag_kind="hist")

### Select Target Column

**Press ▶ to specify the target column.**

In [ ]:
import ipywidgets as widgets
import pandas as pd

# Create a Dropdown widget for column selection
dropdown = widgets.Dropdown(
    options=data.columns.tolist(),
    value=data.columns[0],
    description='Select Column:',
    disabled=False,
    layout=widgets.Layout(width='500px'),
    style={'description_width': '200px'}
)

# Create a Button widget
button = widgets.Button(
    description='Select',
    button_style='',  # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to select the target column as the last column',
    icon='check'  # FontAwesome icon names (without 'fa-')
)

# Create an Output widget for displaying messages
output = widgets.Output()

# Function to handle button click that rearranges the DataFrame
def on_button_clicked(b):
    with output:
        output.clear_output()
        global data
        # Get the selected column name
        selected_column = dropdown.value
        # Reorder the DataFrame columns
        new_columns = [col for col in data.columns if col != selected_column] + [selected_column]
        data = data[new_columns]
        print(f"Column '{selected_column}' has been moved to the last position.")

# Link the button click event to the function
button.on_click(on_button_clicked)

# Display the widgets and output
display(widgets.VBox([dropdown, button, output]))


### Data Preprocessing

**Press ▶ to handle nulls, process categorical values, and normalize data.**

In [ ]:
import pandas as pd
import numpy as np
import ipywidgets as widgets
from IPython.display import display
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

df = data.copy()

# Widgets for selecting operations and methods
fill_method_dropdown = widgets.Dropdown(
    options=[('None', None), ('Zero', 'zero'), ('Mean', 'mean'), ('Median', 'median')],
    value=None,
    description='Fill Method:',
)

remove_nulls_checkbox = widgets.Checkbox(value=False, description='Remove Nulls')
encode_categorical_checkbox = widgets.Checkbox(value=False, description='Encode Categorical')
normalize_data_checkbox = widgets.Checkbox(value=False, description='Normalize Data')

# Button to apply selected operations
apply_button = widgets.Button(description='Apply All', button_style='info')

# Button to display data
show_data_button = widgets.Button(description='Show Data')

# Output area
output = widgets.Output()

def apply_operations():
    global data
    with output:
        output.clear_output()
        # Fill missing values based on selected method
        if fill_method_dropdown.value:
            if fill_method_dropdown.value == 'zero':
                data = data.fillna(0)
            elif fill_method_dropdown.value in ['mean', 'median']:
                # Apply fill method only to numeric columns
                numeric_cols = data.select_dtypes(include=np.number).columns
                for column in numeric_cols:
                    if data[column].sum() != 0:
                        if fill_method_dropdown.value == 'mean':
                            mean_value = data[column].mean()
                            data[column] = data[column].fillna(mean_value)
                        elif fill_method_dropdown.value == 'median':
                            median_value = data[column].median()
                            data[column] = data[column].fillna(median_value)
            print(f"Missing values filled with {fill_method_dropdown.value}.")

        if remove_nulls_checkbox.value:
            data = data.dropna()  # Remove remaining null values
            print("Remaining null values removed.")
        if encode_categorical_checkbox.value:
            # Apply label encoding to categorical columns
            label_encoder = LabelEncoder()
            categorical_cols = data.select_dtypes(include=['object', 'category']).columns
            for col in categorical_cols:
                data[col] = label_encoder.fit_transform(data[col].astype(str))
            print("Categorical data encoded using label encoding.")
        if normalize_data_checkbox.value:
            # Normalize numeric columns using MinMaxScaler
            scaler = MinMaxScaler()
            numeric_cols = data.select_dtypes(include=np.number).columns
            data[numeric_cols] = scaler.fit_transform(data[numeric_cols])
            print("Data normalized.")

def show_data(b):
    with output:
        output.clear_output()
        display(data.head())  # Show the head of the DataFrame

apply_button.on_click(lambda b: apply_operations())
show_data_button.on_click(show_data)

# Layout the widgets
widgets.VBox([
    widgets.Label('Select Fill Method and Operations:'),
    fill_method_dropdown,
    remove_nulls_checkbox,
    encode_categorical_checkbox,
    normalize_data_checkbox,
    apply_button,
    show_data_button,
    output
])


### Split data into training and testing

**Press ▶ to split the data into training and testing sets.**

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import ipywidgets as widgets
from IPython.display import display, clear_output

X = data.iloc[:, :-1]  # all rows, all columns except the last one
y = data.iloc[:, -1]   # all rows, just the last column
# Function to split data and display the shape of the splits

X_train, X_test, y_train, y_test = None, None, None, None

def split_data(button):
    global X_train, X_test, y_train, y_test  # Declare the use of global variables
    test_size = test_size_slider.value
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42)

    # Display the output
    with out:
        clear_output(wait=True)
        print(f"Training set: {X_train.shape[0]} samples")
        print(f"Test set: {X_test.shape[0]} samples")

# Create output widget
out = widgets.Output()


# Create slider for test size
test_size_slider = widgets.FloatSlider(
    value=0.25,  # Default split 75%-25%
    min=0.1,
    max=0.9,
    step=0.05,
    description='Test Size:',
    readout_format='.2f',  # Display format
)

# Create an Apply button
apply_button = widgets.Button(description="Apply Changes")

# Set up button click event to trigger the data split
apply_button.on_click(split_data)

# Organize widgets in a vertical box
widgets_box = widgets.VBox([test_size_slider, apply_button, out])

# Display the widgets
display(widgets_box)

## XGBoost

### Key Parameters in XGBoost

XGBoost (Extreme Gradient Boosting) is a powerful and efficient implementation of the gradient boosting framework. The performance and speed of XGBoost make it a popular choice among data scientists. Key parameters such as `learning_rate`, `n_estimators`, and `max_depth` play crucial roles in model training and accuracy:

#### 1. Learning Rate (`learning_rate`)

The `learning_rate`, also known as `eta`, controls how quickly the model fits the residual error using additional base learners. A lower value makes the model robust to the specific characteristics of the tree but takes longer to compute as more trees are needed. A higher learning rate can lead to rapid convergence but risk overfitting if too high. Typically, a smaller learning rate with more trees could increase accuracy but also computational cost.

#### 2. Number of Estimators (`n_estimators`)

This parameter defines the number of gradient boosted trees to fit. It is equivalent to the number of boosting rounds. More trees can improve accuracy but also risk overfitting, especially when the learning rate is high. It is crucial to balance the number of estimators with the learning rate to achieve the best performance. This parameter interacts with the learning rate (`learning_rate`): a lower learning rate often requires more trees to model all relevant relations and interactions adequately.

#### 3. Maximum Depth (`max_depth`)

`max_depth` sets the maximum depth of each tree. Deeper trees can model more complex patterns by creating more specific rules (i.e., more splits in the decision trees). However, very deep trees can lead to overfitting, especially in noisy data. Limiting the tree depth is a common technique to enhance generalization. 

Properly tuning these parameters is key to building a high-performing XGBoost model. They control the model's complexity and its ability to generalize on unseen data. Overfitting and underfitting are significant concerns that need balancing when adjusting these parameters, often requiring cross-validation to optimize effectively.


**Press ▶ to train the model.**

In [ ]:
import xgboost as xgb
from sklearn.preprocessing import StandardScaler, LabelEncoder
import ipywidgets as widgets
from IPython.display import display

# Assuming X_train and y_train, X_test, y_test are already defined
# Assuming y_train and y_test contain string labels

# Encode target labels with value between 0 and n_classes-1
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Standardize the training and test data
sc = StandardScaler()
X_train_std = sc.fit_transform(X_train)
X_test_std = sc.transform(X_test)

# Initialize global variables for XGBoost classifier and predictions
xgb_clf = None
y_pred = None

# Function to train XGBoost and print accuracy
def train_xgb(button):
    global learning_rate, n_estimators, max_depth, xgb_clf, y_pred
    xgb_clf = xgb.XGBClassifier(learning_rate=learning_rate, n_estimators=n_estimators, max_depth=max_depth, use_label_encoder=False, eval_metric='mlogloss')
    xgb_clf.fit(X_train_std, y_train_encoded)
    train_accuracy = xgb_clf.score(X_train_std, y_train_encoded)
    test_accuracy = xgb_clf.score(X_test_std, y_test_encoded)*100
    y_pred = xgb_clf.predict(X_test_std)
    y_pred = label_encoder.inverse_transform(y_pred)  # Decode the predictions to original labels
    # Display the output
    with out:
        clear_output(wait=True)
        print(f'The accuracy of the xgb classifier is {test_accuracy:.1f}% on test data')
# Create output widget
out = widgets.Output()

# Function to update XGBoost parameters
def update_params(change):
    global learning_rate, n_estimators, max_depth
    learning_rate = learning_rate_slider.value
    n_estimators = n_estimators_slider.value
    max_depth = max_depth_slider.value

# Create sliders for XGBoost parameters
learning_rate_slider = widgets.FloatSlider(
    value=0.1,
    min=0.01,
    max=1.0,
    step=0.01,
    description='Learning Rate:',
    layout=widgets.Layout(width='500px'),
    style={'description_width': '200px'}
)

n_estimators_slider = widgets.IntSlider(
    value=100,
    min=50,
    max=500,
    step=10,
    description='N Estimators:',
    layout=widgets.Layout(width='500px'),
    style={'description_width': '200px'}
)

max_depth_slider = widgets.IntSlider(
    value=3,
    min=1,
    max=10,
    step=1,
    description='Max Depth:',
    layout=widgets.Layout(width='500px'),
    style={'description_width': '200px'}
)

# Create a button to train the model
train_button = widgets.Button(description="Train")

# Link parameter widgets to the update_params function
learning_rate_slider.observe(update_params, 'value')
n_estimators_slider.observe(update_params, 'value')
max_depth_slider.observe(update_params, 'value')

# Set up button click event to trigger the data split
train_button.on_click(train_xgb)

# Organize widgets in a vertical box
widgets_box = widgets.VBox([learning_rate_slider, n_estimators_slider, max_depth_slider, train_button, out])

# Display the widgets
display(widgets_box)


**Press ▶ to display the confusion matrix and the classification report.**

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
print("Confusion Matrix: ")
print(confusion_matrix(y_test,y_pred))

print("Classification Report: ")
print(classification_report(y_test,y_pred))


### Visualization

In classification tasks, the <span style="color:red">red</span> and <span style="color:blue">blue</span> areas typically represent different classes or categories. The <span style="color:red">red</span> area signifies regions where the classifier predicts one class, while the <span style="color:blue">blue</span> area represents regions where it predicts another class, based on the features of the data points in those regions.


**Press ▶ to visualize the decision boundary.**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
import ipywidgets as widgets
from IPython.display import display
import pandas as pd

def plot_decision_regions(X, y, classifier, label_encoder, feature1, feature2, resolution=0.02):
    X = X[[feature1, feature2]]  # Take the selected features for visualization

    # setup marker generator and color map
    markers = ('s', 'x', 'o', '^', 'v')
    colors = ['red', 'blue', 'lightgreen', 'gray', 'cyan']
    cmap = ListedColormap(colors[:len(np.unique(y))])

    # plot the decision surface
    x1_min, x1_max = X.iloc[:, 0].min() - 1, X.iloc[:, 0].max() + 1
    x2_min, x2_max = X.iloc[:, 1].min() - 1, X.iloc[:, 1].max() + 1
    xx1, xx2 = np.meshgrid(np.arange(x1_min, x1_max, resolution),
                           np.arange(x2_min, x2_max, resolution))
    Z = classifier.predict(np.array([xx1.ravel(), xx2.ravel()]).T)
    Z = Z.reshape(xx1.shape)
    plt.contourf(xx1, xx2, Z, alpha=0.4, cmap=cmap)
    plt.xlim(xx1.min(), xx1.max())
    plt.ylim(xx2.min(), xx2.max())
    
    for idx, cl in enumerate(np.unique(y)):
        plt.scatter(x=X.loc[y == cl, feature1], y=X.loc[y == cl, feature2],
                    alpha=0.8, c=cmap(idx),
                    marker=markers[idx], label=label_encoder.inverse_transform([cl])[0])

    plt.xlabel(feature1)
    plt.ylabel(feature2)
    plt.title('XGBoost Classification with Decision Boundaries')
    plt.legend(loc='upper left')

# Encoding the labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Splitting the data
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.3, random_state=42)

# Dropdowns for selecting features
feature_selector1 = widgets.Dropdown(
    options=X.columns,
    description='X Axis:',
    disabled=False,
)

feature_selector2 = widgets.Dropdown(
    options=X.columns,
    description='Y Axis:',
    disabled=False,
)

button = widgets.Button(description="Plot")

output = widgets.Output()

def on_button_click(b):
    with output:
        output.clear_output()
        feature1 = feature_selector1.value
        feature2 = feature_selector2.value
        
        if feature1 and feature2:
            xgb_model = XGBClassifier()
            xgb_model.fit(X_train[[feature1, feature2]], y_train)
            plt.figure(figsize=(10, 5))
            plot_decision_regions(X_test, y_test, xgb_model, label_encoder, feature1, feature2)
            plt.show()
        else:
            print("Please select both features")

button.on_click(on_button_click)

display(feature_selector1, feature_selector2, button, output)


### <center>[🏠 Home](../../../../welcomePage.ipynb)</center>